In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString

# Define system parameters
K = np.array([[0.765, 0.087], [0.002, 0.810]])

# Define state and input constraints
x_min, x_max = -5, 5
u_min, u_max = -1, 1

# Function to find intersection of two lines defined by points
def line_intersection(line1, line2):
    line1 = LineString(line1)
    line2 = LineString(line2)
    int_pt = line1.intersection(line2)
    return [int_pt.x, int_pt.y]

# Create boundaries of all constraints
constraints = []

# State boundaries
state_lines = [
    [[-5, -5], [5, -5]],  # Bottom
    [[5, -5], [5, 5]],    # Right
    [[5, 5], [-5, 5]],    # Top
    [[-5, 5], [-5, -5]]   # Left
]
constraints.extend(state_lines)

# Input constraints from first row of K: -K[0,0]*x1 - K[0,1]*x2 = ±1
# These create sloped lines
x1_points = np.array([-20, 20])  # Make lines long enough
y1_upper = (u_max + K[0,0]*x1_points)/K[0,1]  # -0.765x1 - 0.087x2 = -1 → x2 = (1 - 0.765x1)/0.087
y1_lower = (u_min + K[0,0]*x1_points)/K[0,1]  # -0.765x1 - 0.087x2 = 1 → x2 = (-1 - 0.765x1)/0.087

constraints.append([[x1_points[0], y1_upper[0]], [x1_points[1], y1_upper[1]]])
constraints.append([[x1_points[0], y1_lower[0]], [x1_points[1], y1_lower[1]]])

# Input constraints from second row of K: -K[1,0]*x1 - K[1,1]*x2 = ±1
# These create almost horizontal lines (since K[1,0] is very small)
y2_upper = (u_max + K[1,0]*x1_points)/K[1,1]  # -0.002x1 - 0.810x2 = -1 → x2 = (1 - 0.002x1)/0.810
y2_lower = (u_min + K[1,0]*x1_points)/K[1,1]  # -0.002x1 - 0.810x2 = 1 → x2 = (-1 - 0.002x1)/0.810

constraints.append([[x1_points[0], y2_upper[0]], [x1_points[1], y2_upper[1]]])
constraints.append([[x1_points[0], y2_lower[0]], [x1_points[1], y2_lower[1]]])

# Find all valid intersections
intersections = []
for i in range(len(constraints)):
    for j in range(i+1, len(constraints)):
        try:
            pt = line_intersection(constraints[i], constraints[j])
            # Check if point satisfies all constraints
            x1, x2 = pt
            
            if (x_min <= x1 <= x_max and x_min <= x2 <= x_max and
                u_min <= -K[0,0]*x1 - K[0,1]*x2 <= u_max and
                u_min <= -K[1,0]*x1 - K[1,1]*x2 <= u_max):
                intersections.append(pt)
        except:
            pass  # Lines may be parallel or not intersect

# Convert to numpy array
if intersections:
    intersections = np.array(intersections)

    # Order points to form a polygon
    center = np.mean(intersections, axis=0)
    angles = np.arctan2(intersections[:,1] - center[1], 
                        intersections[:,0] - center[0])
    sorted_indices = np.argsort(angles)
    vertices = intersections[sorted_indices]

    # Create plot
    plt.figure(figsize=(10, 10))

    # Plot state constraints
    plt.axhline(y=x_max, color='r', linestyle='--', label='State Constraints')
    plt.axhline(y=x_min, color='r', linestyle='--')
    plt.axvline(x=x_max, color='r', linestyle='--')
    plt.axvline(x=x_min, color='r', linestyle='--')

    # Plot input constraints
    x_plot = np.linspace(-5, 5, 100)
    
    # First input constraint
    y_plot_upper = (u_max + K[0,0]*x_plot)/K[0,1]
    y_plot_lower = (u_min + K[0,0]*x_plot)/K[0,1]
    plt.plot(x_plot, y_plot_upper, 'g--', label='Input Constraint 1')
    plt.plot(x_plot, y_plot_lower, 'g--')
    
    # Second input constraint
    y_plot_upper = (u_max + K[1,0]*x_plot)/K[1,1]
    y_plot_lower = (u_min + K[1,0]*x_plot)/K[1,1]
    plt.plot(x_plot, y_plot_upper, 'b--', label='Input Constraint 2')
    plt.plot(x_plot, y_plot_lower, 'b--')

    # Plot X0 set
    vertices_closed = np.vstack([vertices, vertices[0]])
    plt.fill(vertices_closed[:,0], vertices_closed[:,1], 'lightblue', alpha=0.5)
    plt.plot(vertices_closed[:,0], vertices_closed[:,1], 'k-', linewidth=2, label='X0 Boundary')
    plt.scatter(vertices[:,0], vertices[:,1], color='red', s=80, label='Vertices')

    # Print vertices
    print("Vertices of X0:")
    for i, vertex in enumerate(vertices):
        print(f"Vertex {i+1}: ({vertex[0]:.4f}, {vertex[1]:.4f})")

    plt.title('Terminal Constraint Set X0')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.grid(True)
    plt.legend()
    plt.axis('equal')
    plt.xlim([-5.5, 5.5])
    plt.ylim([-5.5, 5.5])

    plt.show()